In [1]:
from datasets import Mydataset_plip2
from torch.utils.data import DataLoader
from load_model import model_fusion
import torch
import pandas as pd
import torch.nn.functional as F
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import json
from operator import itemgetter

In [2]:
data = pd.read_csv("internal_corhort.csv")

In [3]:
data.head(1) # the protoype  embedding get from step1

,name,case,kind,description,disease,prototype,pro_num
0,a21-1884_brain0.csv,a21-1884,brain,Scattered punctate hemorrhage is observed unde...,The hemorrhage under the scalp,"{830: 1259, 555: 567, 786: 504, 535: 497, 857:...",80


In [4]:
dd = Mydataset_plip2("internal_corhort.csv", randomm=False)
trainloader = DataLoader(dd, batch_size=1, shuffle=False,drop_last=False)

In [5]:
model = model_fusion(depth=2,noise_ratio=0.0, gate=True,num_em=True)
model.load_state_dict(torch.load("fusion_checkpoint.pth",map_location="cpu"))

<All keys matched successfully>

In [6]:
model.eval()
model.cuda()

fusionblock2(
  (text_model): CLIPModel(
    (shared_parameters): ModuleDict()
    (text_model): CLIPTextTransformer(
      (invertible_adapters): ModuleDict()
      (embeddings): CLIPTextEmbeddings(
        (token_embedding): Embedding(49408, 512)
        (position_embedding): Embedding(77, 512)
      )
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0): CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(
                in_features=512, out_features=512, bias=True
                (loras): ModuleDict()
              )
              (v_proj): Linear(
                in_features=512, out_features=512, bias=True
                (loras): ModuleDict()
              )
              (q_proj): Linear(
                in_features=512, out_features=512, bias=True
                (loras): ModuleDict()
              )
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
              (prefix_tuning): PrefixTuni

In [7]:
# If you have a set of candidate diagnoses, we can use the diagnosis of data as an example of a candidate in this context.
for it, (tokens_disease,tokens_description,img,img_num) in enumerate(trainloader):
    tokens_description["input_ids"] = tokens_description["input_ids"].squeeze(1).cuda(non_blocking=True)
    tokens_description["attention_mask"] = tokens_description["attention_mask"].squeeze(1).cuda(non_blocking=True)
    tokens_disease["input_ids"] = tokens_disease["input_ids"].squeeze(1).cuda(non_blocking=True)
    tokens_disease["attention_mask"] = tokens_disease["attention_mask"].squeeze(1).cuda(non_blocking=True)
    _, _, _, img_text_out, disease, _ = model(img_p=img.cuda(non_blocking=True),text=tokens_description,img_num=img_num.cuda(non_blocking=True),disease=tokens_disease)
    score = img_text_out.cpu().squeeze() @ disease.cpu().squeeze().t()

    break

In [8]:
print(score) # Return the similarity between the fused embedding of the multimodal input and the target diagnosis!

tensor(0.9173, grad_fn=<DotBackward0>)


In [9]:
# If you do not have any candidate diagnoses, as an exsample, we provide commonly used diagnoses in forensic pathology here.

In [9]:
with open("disease.txt", "r") as file:
    my_list = [line.strip() for line in file]
tokenizer = CLIPProcessor.from_pretrained("vinid/plip").tokenizer
token_disease = tokenizer(my_list, padding="max_length", max_length=77, return_tensors="pt")
token_disease["input_ids"] = token_disease["input_ids"].cuda(non_blocking=True)
token_disease["attention_mask"] = token_disease["attention_mask"].cuda(non_blocking=True)
disease_model = CLIPModel.from_pretrained("vinid/plip")
disease_model.eval()
disease_model.cuda()
disease_rep = disease_model.get_text_features(**token_disease)
disease_rep = F.normalize(disease_rep, dim=-1).cpu()
for it, (tokens_disease,tokens_description,img,img_num) in enumerate(trainloader):
    score_list=[]
    model.eval()
    with torch.no_grad():
        tokens_description["input_ids"] = tokens_description["input_ids"].squeeze(1).cuda(non_blocking=True)
        tokens_description["attention_mask"] = tokens_description["attention_mask"].squeeze(1).cuda(non_blocking=True)
        for ind, (t_disease,t_attention) in enumerate(zip(token_disease["input_ids"],token_disease["attention_mask"])):
            _, _, _, img_text_out, disease, attention_score = model(img_p=img.cuda(non_blocking=True),
                                                                           text=tokens_description,
                                                                           img_num=img_num.cuda(non_blocking=True),
                                                                           disease={"input_ids":t_disease.unsqueeze(0),"attention_mask":t_attention.unsqueeze(0)})
            score = img_text_out.cpu().squeeze() @ disease_rep[ind].t()
            score_list.append(score)

        score_array = np.array(score_list)


        disease_name_max = [my_list[i] for i in score_array.argsort()[-1:][::-1].tolist()]  # top1
        disease_name_top2 = [my_list[i] for i in score_array.argsort()[-2:][::-1].tolist()] # top2 
        disease_name_top3 = [my_list[i] for i in score_array.argsort()[-3:][::-1].tolist()]  # top3 
    break


In [10]:
disease_name_max # best answer!

['The hemorrhage under the scalp']

In [11]:
disease_name_top2

['The hemorrhage under the scalp', 'Cerebral hemorrhage']

In [12]:
disease_name_top3

['The hemorrhage under the scalp',
 'Cerebral hemorrhage',
 'Cerebral congestion']

In [13]:
data.loc[0,"disease"] # the groudtruth 

'The hemorrhage under the scalp'

In [14]:
with open("disease.json","r", encoding='utf-8') as f:
    di_dic = json.load(f)
ll_iou = []
ll_recall = []
ll_precision = []
for it, (tokens_disease,tokens_description,img,img_num) in enumerate(trainloader):
    print(it)
    score_list=[]
    model.eval()
    with torch.no_grad():
        tokens_description["input_ids"] = tokens_description["input_ids"].squeeze(1).cuda(non_blocking=True)
        tokens_description["attention_mask"] = tokens_description["attention_mask"].squeeze(1).cuda(non_blocking=True)
        for ind, (t_disease,t_attention) in enumerate(zip(token_disease["input_ids"],token_disease["attention_mask"])):
            _, _, _, img_text_out, disease, attention_score = model(img_p=img.cuda(non_blocking=True),
                                                                           text=tokens_description,
                                                                           img_num=img_num.cuda(non_blocking=True),
                                                                           disease={"input_ids":t_disease.unsqueeze(0),"attention_mask":t_attention.unsqueeze(0)})
            score = img_text_out.cpu().squeeze() @ disease_rep[ind].t()
            score_list.append(score)

        score_array = np.array(score_list)


        disease_name_max = [my_list[i] for i in score_array.argsort()[-1:][::-1].tolist()]  # top1
        disease_name_top2 = [my_list[i] for i in score_array.argsort()[-2:][::-1].tolist()] # top2 
        disease_name_top3 = [my_list[i] for i in score_array.argsort()[-3:][::-1].tolist()]  # top3 
        
        disease_name = [my_list[i] for i, ss in enumerate(score_list) if ss >0.881] # predict disease under the thresholds
        diagnois = [i.strip().capitalize() for i in data.loc[it, "disease"].split("/")]  ## label
        
        if not disease_name:
            disease_name = disease_name_max
        # Combining similar diagnoses based on the judgement of the Senior Forensic Pathologist to avoid confusion.    
        print("Predict disease:","/".join(disease_name))
        if len(disease_name) > 1:
            disease_name_num = list(itemgetter(*disease_name)(di_dic))
        else:
            disease_name_num = [di_dic[disease_name[0]]]

        if len(diagnois) > 1:
            diagnois_num = list(itemgetter(*diagnois)(di_dic))
        else:
            diagnois_num = [di_dic[diagnois[0]]]
        
        iou = len(list(set(disease_name_num) & set(diagnois_num))) / len(list(set(disease_name_num) | set(diagnois_num)))
        recall = len(list(set(disease_name_num) & set(diagnois_num))) / len(set(diagnois_num))
        precision = len(list(set(disease_name_num) & set(diagnois_num))) / len(set(disease_name_num))
        ll_iou.append(iou)
        ll_precision.append(precision)
        ll_recall.append(recall)
    #break
# Here we print the forensic pathology diagnoses for each sample of the internal cohort!

0
Predict disease: The hemorrhage under the scalp
1
Predict disease: The hemorrhage under the scalp
2
Predict disease: Adrenal lipids decreased/Adrenal congestion/Adrenal gland infections/Dense adrenal glands/Chronic focal inflammation of the adrenal gland
3
Predict disease: Viral myocarditis/Myocarditis/Interstitial myocarditis
4
Predict disease: Interstitial myocarditis
5
Predict disease: Viral myocarditis/Suppurative myocarditis/Myocarditis/Interstitial myocarditis
6
Predict disease: Gastrointestinal congestion/Inflammatory cells infiltrate gastrointestinal tissue/Gastrointestinal tissue autolysis
7
Predict disease: Renal fibrosis/A small amount of glomerular fibrosis/Mild renal autolysis/Mild glomerular fibrosis/Glomerular fibrosis/Congestion of kidney/Partial glomerular fibrosis/Renal autolysis/Renal congestion
8
Predict disease: Congestion of liver
9
Predict disease: Pulmonary edema/Pulmonary congestion
10
Predict disease: Pancreatic autolysis
11
Predict disease: Hyalinosis of ce

Predict disease: Myocardial interstitial lymphocyte infiltration
102
Predict disease: Diffuse suppurative peritonitis
103
Predict disease: Mild renal autolysis/Congestion of kidney/Renal autolysis/Renal congestion
104
Predict disease: The liver surface oozes cellulose
105
Predict disease: Pulmonary thrombosis
106
Predict disease: Congestion of the pancreas/Pancreatic congestion/Pancreatic autolysis/Congestion of pancreas
107
Predict disease: Central splenic arteriosclerosis
108
Predict disease: Cerebral congestion
109
Predict disease: Cerebral congestion
110
Predict disease: Coronary atherosclerosis
111
Predict disease: Coronary atherosclerosis
112
Predict disease: Gastrointestinal congestion
113
Predict disease: Glomerular fibrosis/Congestion of kidney/Renal autolysis/Renal congestion
114
Predict disease: Congestion of liver/Liver inflammatory cell infiltration/Hepatic steatosis
115
Predict disease: Pulmonary edema/Pulmonary congestion
116
Predict disease: Congestion of the pancreas/P

Predict disease: Congestion of liver/Severe fatty liver
205
Predict disease: Pulmonary edema/Pulmonary congestion
206
Predict disease: Congestion of the pancreas/Pancreatic autolysis/Congestion of pancreas
207
Predict disease: Hyalinosis of central splenic artery/Congestion of spleen
208
Predict disease: Cerebral congestion/Cerebral contusion
209
Predict disease: Adrenal lipids decreased/Adrenal congestion/Adrenal gland infections/Dense adrenal glands
210
Predict disease: Myocardial congestion
211
Predict disease: Gastrointestinal tissue autolysis
212
Predict disease: A small amount of glomerular fibrosis/Mild renal autolysis/Mild glomerular fibrosis/Glomerular fibrosis/Congestion of kidney/Renal autolysis/Renal congestion
213
Predict disease: Congestion of liver/Hepatic steatosis
214
Predict disease: Pulmonary edema/Pulmonary congestion
215
Predict disease: Congestion of the pancreas/Pancreatic autolysis/Congestion of pancreas
216
Predict disease: Central splenic arteriosclerosis
217


Predict disease: Adrenal lipids decreased/Adrenal congestion/Adrenal gland infections/Dense adrenal glands
302
Predict disease: Myocardial interstitial inflammation cell infiltration
303
Predict disease: Gastrointestinal congestion/Inflammatory cells infiltrate gastrointestinal tissue/Lymphocyte infiltration in gastrointestinal tissue/Infiltration of lymphocytes in the gastrointestinal mucosa
304
Predict disease: Glomerular fibrosis/Congestion of kidney/Renal autolysis/Renal congestion
305
Predict disease: Congestion of liver
306
Predict disease: Pulmonary edema/Pulmonary congestion
307
Predict disease: Congestion of the pancreas/Pancreatic autolysis/Congestion of pancreas
308
Predict disease: Splenic congestion/Hyalinosis of central splenic artery/Congestion of spleen
309
Predict disease: Cerebral congestion/Cerebral edema
310
Predict disease: Myocardial congestion
311
Predict disease: Gastrointestinal congestion/Gastrointestinal autolysis/Gastrointestinal tissue autolysis
312
Predict

Predict disease: Congestion of liver
404
Predict disease: Pulmonary edema/Pulmonary exudation/Pulmonary congestion/Pulmonary infection
405
Predict disease: Congestion of the pancreas/Pancreatic congestion/Pancreatic autolysis/Congestion of pancreas
406
Predict disease: Central splenic arteriosclerosis
407
Predict disease: Cerebral congestion
408
Predict disease: Myocardial congestion
409
Predict disease: Gastrointestinal congestion/Gastrointestinal tissue autolysis
410
Predict disease: Mild renal autolysis/Congestion of kidney/Renal autolysis/Renal congestion
411
Predict disease: Congestion of liver
412
Predict disease: Pulmonary edema/Pulmonary exudation/Pulmonary congestion/Severe pulmonary infection/Pulmonary infection/Focal pulmonary infection/Severe lung infection
413
Predict disease: Congestion of the pancreas/Pancreatic congestion/Pancreatic autolysis/Congestion of pancreas
414
Predict disease: Central splenic arteriosclerosis
415
Predict disease: Central splenic arteriosclerosi

...

Predict disease: Splenic congestion/Congestion of spleen
1030
Predict disease: Cerebral congestion/Cerebral edema
1031
Predict disease: Adrenal lipids decreased/Adrenal congestion/Adrenal gland infections/Dense adrenal glands/No change
1032
Predict disease: Myocardial congestion
1033
Predict disease: Gastrointestinal congestion/Gastrointestinal tissue autolysis
1034
Predict disease: A small amount of glomerular fibrosis/Mild renal autolysis/Glomerular fibrosis/Congestion of kidney/Renal autolysis/Renal congestion
1035
Predict disease: Hepatic congestion/Congestion of liver/Hepatic lipidosis
1036
Predict disease: Pulmonary edema/Pulmonary exudation/Pulmonary atelectasis/Pulmonary congestion/Severe pulmonary infection/Pulmonary autolysis/Pulmonary infection/Focal pulmonary infection
1037
Predict disease: Congestion of the pancreas/Pancreatic autolysis/Congestion of pancreas
1038
Predict disease: Congestion of spleen/Splenic lymphocytopenia
1039
Predict disease: Cerebral congestion
1040
P

Predict disease: Pulmonary edema/Pulmonary congestion/Pulmonary infection
1125
Predict disease: Pancreatic autolysis
1126
Predict disease: Central splenic arteriosclerosis
1127
Predict disease: Cerebral congestion
1128
Predict disease: Cardiac hypertrophy
1129
Predict disease: Arteriolar sclerosis of the kidney/Congestion of kidney/Mild small arteriosclerosis kidney/Small arteriosclerosis kidney/Renal autolysis
1130
Predict disease: Pulmonary edema/Pulmonary congestion
1131
Predict disease: Cerebral contusion
1132
Predict disease: Cardiac congestion
1133
Predict disease: Adrenal congestion
1134
Predict disease: Mild renal autolysis/Congestion of kidney/Renal autolysis
1135
Predict disease: Hepatic lipidosis
1136
Predict disease: Lobular pneumonia
1137
Predict disease: Pancreatic autolysis
1138
Predict disease: Central splenic arteriosclerosis
1139
Predict disease: Cerebral congestion
1140
Predict disease: Coronary atherosclerosis
1141
Predict disease: Mild renal autolysis/Congestion of

Predict disease: A small amount of glomerular fibrosis/Mild renal autolysis/Mild glomerular fibrosis/Glomerular fibrosis/Congestion of kidney/Partial glomerular fibrosis/Renal autolysis/Renal congestion
1236
Predict disease: Myocardial interstitial congestion
1237
Predict disease: Hyalinization of the central splenic artery/Congestion of spleen
1238
Predict disease: Congestion of the pancreas/No abnormalities/Pancreatic autolysis/Congestion of pancreas
1239
Predict disease: Pulmonary edema/Pulmonary exudation/Pulmonary congestion/Severe pulmonary infection/Pulmonary infection
1240
Predict disease: Liver lymphocyte infiltration/Lymphocyte infiltration of liver/Congestion of liver/Hepatic steatosis
1241
Predict disease: A small amount of glomerular fibrosis/Mild renal autolysis/Mild glomerular fibrosis/Glomerular fibrosis/Congestion of kidney/Partial glomerular fibrosis/Renal autolysis/Renal congestion
1242
Predict disease: Gastrointestinal congestion/Gastrointestinal bleeding/Gastrointe

Predict disease: Gastrointestinal congestion/Gastrointestinal autolysis
1317
Predict disease: Myocardial congestion
1318
Predict disease: Adrenal congestion
1319
Predict disease: Cerebral congestion
1320
Predict disease: Congestion of spleen
1321
Predict disease: Congestion of the pancreas/Pancreatic autolysis/Congestion of pancreas
1322
Predict disease: Pulmonary edema/Pulmonary congestion
1323
Predict disease: Congestion of liver
1324
Predict disease: A small amount of glomerular fibrosis/Mild renal autolysis/Mild glomerular fibrosis/Glomerular fibrosis/Congestion of kidney/Partial glomerular fibrosis/Renal autolysis/Renal congestion
1325
Predict disease: Gastrointestinal congestion/Gastrointestinal autolysis
1326
Predict disease: Myocardial congestion
1327
Predict disease: Myocardial congestion
1328
Predict disease: Adrenal congestion/Dense adrenal glands
1329
Predict disease: Cerebral congestion/Cerebral edema
1330
Predict disease: Congestion of spleen
1331
Predict disease: Congest

Predict disease: Congestion of the pancreas/Pancreatic autolysis/Congestion of pancreas
1409
Predict disease: Pulmonary edema/Pulmonary exudation/Pulmonary congestion/Pulmonary emphysema/Pulmonary emphysema of lung
1410
Predict disease: Pulmonary edema/Pulmonary exudation/Pulmonary congestion
1411
Predict disease: Congestion of liver
1412
Predict disease: A small amount of glomerular fibrosis/Mild renal autolysis/Mild glomerular fibrosis/Glomerular fibrosis/Congestion of kidney/Partial glomerular fibrosis/Renal autolysis/Renal congestion
1413
Predict disease: Gastrointestinal congestion/Gastrointestinal autolysis/Gastrointestinal tissue autolysis
1414
Predict disease: Myocardial interstitial fibrosis
1415
Predict disease: Cerebral congestion/Cerebral edema
1416
Predict disease: Anemia of spleen/Hyalinization of the central splenic artery/Congestion of spleen
1417
Predict disease: Congestion of the pancreas/Pancreatic autolysis/Congestion of pancreas
1418
Predict disease: Pulmonary edem

In [15]:
sum(ll_iou)/len(ll_iou) # the final IOU

0.6823241770864095

In [16]:
sum(ll_precision)/len(ll_precision)

0.8139969150996033

In [17]:
sum(ll_recall)/len(ll_recall)

0.8230071215253854